In [78]:
import openai
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
from dotenv import load_dotenv

import time
import plotly.express as px
import os
import pandas as pd
import numpy as np

In [47]:
load_dotenv()

True

In [48]:
openai.api_key = os.getenv("OPENAPI_KEY")

In [49]:
word_list = [
    "France",
    "Germany",
    "Aldi",
    "Lidl",
    "Cheese",
    "Porsche",
    "Cyberpunk",
    "Winnie the Poo",
    "John Cena",
    "Moon",
    "Programming",
    "REST API",
    "Feng Shui",
    "Qi",
    "Red",
    "Meth",
    "Methane",
    "Goodman",
    "Ellen Ripley",
    "Meat",
    "T800",
    "Stan Winston",
]

In [5]:
word_embeddings = []
for word in word_list:
    word_embeddings.append(get_embedding(word, engine="text-embedding-ada-002"))

In [50]:
# Make term and emebddings df
df = pd.DataFrame({"Word List": word_list, "Embeddings": word_embeddings})

In [52]:
# Export word embeddings to storage for later use. Saves API calls. 
df.to_csv("word_embeddings.csv")

In [53]:
df

,Word List,Embeddings
0,France,"[0.03075573965907097, -0.005854075308889151, -..."
1,Germany,"[0.019835034385323524, -0.026546243578195572, ..."
2,Aldi,"[-0.007395139895379543, -0.004052354488521814,..."
3,Lidl,"[-0.0035336303990334272, 0.0005827172426506877..."
4,Cheese,"[0.010652019642293453, 0.0009668035199865699, ..."
5,Porsche,"[0.005553688388317823, -0.003836322110146284, ..."
6,Cyberpunk,"[0.0036292874719947577, -0.017004676163196564,..."
7,Winnie the Poo,"[0.0006941117462702096, -0.022078173235058784,..."
8,John Cena,"[-0.013457232154905796, 0.004303531255573034, ..."
9,Moon,"[0.018849844112992287, -0.008479586802423, 0.0..."


In [59]:
df["Embeddings"].loc[0]

[0.03075573965907097,
 -0.005854075308889151,
 -0.003945284057408571,
 -0.03965447098016739,
 0.0015954204136505723,
 0.012218954041600227,
 -0.014020208269357681,
 -0.023429743945598602,
 0.0034176779445260763,
 -0.023752357810735703,
 0.0017609273781999946,
 0.009839685633778572,
 0.004301501903682947,
 -0.025069691240787506,
 0.00024321969249285758,
 0.00456026429310441,
 0.04858008772134781,
 -0.010478190146386623,
 0.02699192613363266,
 -0.020674094557762146,
 -0.001012365217320621,
 0.012104695662856102,
 0.009402814321219921,
 0.016493571922183037,
 0.015673598274588585,
 0.0040393793024122715,
 0.010048040188848972,
 -0.029062023386359215,
 0.002999289659783244,
 -0.0060422662645578384,
 0.004829108249396086,
 -0.0009149093530140817,
 -0.023806124925613403,
 -0.04038035124540329,
 -0.009060039184987545,
 -0.02174947038292885,
 -0.010834407992661,
 -0.013623664155602455,
 0.015566060319542885,
 -0.015794578939676285,
 0.021641932427883148,
 0.017179124057292938,
 0.0048291082493

In [158]:
def embed(term):
    return get_embedding(term, engine="text-embedding-ada-002")


def check_similarity(search, compare):
    search_embedding = embed(search)
    compare_embedding = embed(compare)
    return cosine_similarity(search_embedding, compare_embedding)


def calculate_similarity(df, search_term, pprint=True):
    embedding = embed(search_term)
    df['similarities'] = df["Embeddings"].apply(lambda x: cosine_similarity(x, embedding))
    res = df.sort_values('similarities', ascending=False)
    res_20 = res.head(20)
    plotSimilarities(res_20, search_term, res_20.columns[0])
    return res_20


def plotSimilarities(df, search_term, hover_name):
    fig = px.scatter(
        df, 
        x="similarities", 
        y=[0]*len(df["similarities"]), 
        hover_name=hover_name,
        title="For Search Term:" + search_term    )
    #fig.update_layout(xaxis=dict(range=[0, 1]))
    fig.show()

In [159]:
df

,Word List,Embeddings,similarities
0,France,"[0.03075573965907097, -0.005854075308889151, -...",0.774806
1,Germany,"[0.019835034385323524, -0.026546243578195572, ...",0.809052
2,Aldi,"[-0.007395139895379543, -0.004052354488521814,...",0.788254
3,Lidl,"[-0.0035336303990334272, 0.0005827172426506877...",0.784039
4,Cheese,"[0.010652019642293453, 0.0009668035199865699, ...",0.776615
5,Porsche,"[0.005553688388317823, -0.003836322110146284, ...",0.792697
6,Cyberpunk,"[0.0036292874719947577, -0.017004676163196564,...",0.786766
7,Winnie the Poo,"[0.0006941117462702096, -0.022078173235058784,...",0.757311
8,John Cena,"[-0.013457232154905796, 0.004303531255573034, ...",0.760426
9,Moon,"[0.018849844112992287, -0.008479586802423, 0.0...",0.749404


In [143]:
feng_shui = "Das Layout deiner Wohnung lässt die Energie fließen. Das Sofa ist ideal in der Command Position platziert"
arnie = "Arnold Schwarzenegger"
t800 = "T-800"
wrestling = "Wrestling"
major_jaune = "Le major jaune"
rammstein = "Rammstein"

In [154]:
res = calculate_similarity(df, rammstein)

In [110]:
#######################################################

# Import Uwe Seeler Wikipedia Article

In [36]:
import re
import csv

# Open the input text file
with open("uwe_seeler_wikipedia.txt", "r") as f:
    text = f.read()

# Use regular expressions to split the text into sentences
sentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", text)

# Open the output csv file
with open("uwe_seeler_wikipedia.csv", "w") as f:
    writer = csv.writer(f)

    # Write the header row
    writer.writerow(["Text"])

    # Initialize a variable to store the current sentence
    current_sentence = ""

    # Iterate over each sentence
    for sentence in sentences:
        # Check the length of the sentence
        if len(sentence.split()) < 5:
            # Append the current sentence with the next sentence
            current_sentence += sentence
        else:
            # Write the current sentence to the csv file
            writer.writerow([current_sentence + sentence])
            current_sentence = ""


In [37]:
uweDF = pd.read_csv('uwe_seeler_wikipedia.csv')

In [80]:
def spread_api_calls(text):
    print(text)
    embedded_text = embed(text)
    time.sleep(1)
    return embedded_text

In [81]:
uweDF["embeddings"] = uweDF["Text"].apply(lambda x: spread_api_calls(x))

Uwe Seeler (* 5.November 1936 in Hamburg; † 21.
Juli 2022 in Norderstedt) war ein deutscher Fußballspieler und späterer ‑funktionär.
Er galt in seiner aktiven Zeit als einer der besten Mittelstürmer der Welt.
Seeler spielte während seiner gesamten Karriere beim Hamburger SV und gilt als „größter HSVer aller Zeiten“ sowie als eine der größten Legenden des deutschen Fußballs.
In der Saison 1963/64 war er der erste Torschützenkönig der Bundesliga.
Als Kapitän der deutschen Nationalmannschaft wurde er 1966 Vizeweltmeister und erreichte bei der Weltmeisterschaft 1970 den dritten Platz.
Wegen seiner Verdienste um den deutschen Fußball ernannte ihn der DFB 1972 als zweiten Spieler überhaupt zum Ehrenspielführer der Nationalelf.
Ab 2003 war Seeler Ehrenbürger seiner Heimatstadt.
Inhaltsverzeichnis

    1 Jugend
    2 Karriere im Verein
    3 Karriere in der Nationalmannschaft
    4 Erfolge und Auszeichnungen
        4.1 Erfolge mit der Nationalmannschaft
        4.2 Erfolge mit dem HSV
       

Den Zeitpunkt des Abschieds vom Leistungssport schätzte Seeler auch Jahre später als „genau richtig“ ein.
Diesen Schritt habe er langfristig vorbereitet, die Doppelbelastung mit Fußball und Beruf habe ihn „sehr beansprucht“, der Abschied vom Fußball sei ihm „verhältnismäßig leichtgefallen“, so Seeler im Jahr 1986 gegenüber dem Hamburger Abendblatt.[14]

Diesen Rücktritt unterbrach er noch ein Mal, als er auf Bitten des Sportartikelherstellers Adidas, mit dem er beruflich in Verbindung stand, am 23.
April 1978 ein Gastspiel beim Cork Celtic in der ersten irischen Liga bestritt.
Seeler war nach eigenen Angaben nicht bewusst, dass es sich dabei um ein Punktspiel gehandelt hat, weil ihm nicht bekannt war, dass in der irischen Liga auch für Punktspiele Gastspieler angemeldet werden konnten.
Bei der 2:6-Niederlage gegen die Shamrock Rovers erzielte er beide Tore.[15]
Karriere in der Nationalmannschaft
Seeler im Trainingslager der Nationalmannschaft in der Sportschule Malente (1965)
Uwe Seele

Das Foto zeigt das Label in einem weißen Herrenhemd.

Nach seiner Zeit als aktiver Sportler arbeitete Seeler als Inhaber eines Unternehmens („Uwe-Seeler-Moden“),[24] das Sportbekleidung anbot, sowie als Repräsentant für Adidas (eine Tätigkeit, die er auf Vermittlung von Sepp Herberger schon seit 1961 ausübte und für die er um 1970 ca.
zwei Tage pro Woche aufwendete.
Zu dieser Zeit war er auch Inhaber einer Tankstelle.[3]) In seiner Tätigkeit für Adidas legte Seeler mit seinem Wagen jährlich teils rund 40.000 Kilometer zurück.[24]

1980 gründete Seeler unter anderem mit Franz Beckenbauer und Willi Holdorf den „Schneeforscher e.
V.“ Die Mitglieder dieses Freundeskreises kamen in den folgenden Jahren stets Anfang Dezember in Obertauern zum gemeinsamen Skiurlaub zusammen.[25]

Daneben betätigte er sich in verschiedenen ehrenamtlichen Positionen, unter anderem im Rahmen der Uwe Seeler Traditionself, einer Mannschaft mit mehr als 60 ehemaligen Profis, die in wechselnder Zusammensetzung etwa 

In [116]:
uweDF.rename(columns={'embeddings': 'Embeddings'}, inplace=True)

In [118]:
# Save for later use without using the API
uweDF.to_csv("uwe_seeler_embeddings.csv")

In [155]:
uweDF

,Text,Embeddings,similarities
0,Uwe Seeler (* 5.November 1936 in Hamburg; † 21.,"[-0.008904125541448593, -0.013433169573545456,...",0.794575
1,Juli 2022 in Norderstedt) war ein deutscher Fu...,"[-0.010335334576666355, -0.00362282688729465, ...",0.834365
2,Er galt in seiner aktiven Zeit als einer der b...,"[-0.009197586216032505, 0.005072151776403189, ...",0.808211
3,Seeler spielte während seiner gesamten Karrier...,"[-0.0018050437793135643, 0.005551062989979982,...",0.832411
4,In der Saison 1963/64 war er der erste Torschü...,"[-0.00017848526476882398, -0.00604895735159516...",0.826378
...,...,...,...
126,FC Union Berlin.\nDer ehemalige Bürgerschafts-...,"[0.01595756597816944, -0.0019251321209594607, ...",0.810633
127,"Dessen Söhne, Christian Seeler, ehemaliger Int...","[0.018253158777952194, -0.014568915590643883, ...",0.765818
128,Juli 2022 starb Seeler im Alter von 85 Jahren ...,"[-0.006996626034379005, 0.013396690599620342, ...",0.785010
129,a.ein Banner mit der Aufschrift Loyal und besc...,"[-0.005660532973706722, 0.014033057726919651, ...",0.770572


In [175]:
res_20_df = calculate_similarity(uweDF, "Kleidung")


In [176]:
res_20_df

,Text,Embeddings,similarities
102,Das Foto zeigt das Label in einem weißen Herre...,"[0.0005896095535717905, -0.0019138663774356246...",0.829639
12,Er wuchs in der Frickestraße auf und besuchte ...,"[0.0013073411537334323, -0.003077553818002343,...",0.787177
104,zwei Tage pro Woche aufwendete.,"[-0.004541965201497078, 0.009552310220897198, ...",0.782687
11,Er und seine Geschwister seien „zu absoluter K...,"[0.011969451792538166, 0.01458656694740057, 0....",0.781159
29,Sein langjähriger Freund Adi Dassler bot Seele...,"[0.005290338769555092, -0.003337639383971691, ...",0.780555
30,Gemeinsam mit seiner Frau Ilka habe er lange ü...,"[0.019009185954928398, 0.0012367324670776725, ...",0.780477
40,Doch sechs Monate später stand er wieder auf d...,"[-0.021779021248221397, -0.009096485562622547,...",0.776497
97,Minute im gleichzeitig stattfindenden Spiel Br...,"[-0.023267250508069992, 0.009005026891827583, ...",0.775968
126,FC Union Berlin.\nDer ehemalige Bürgerschafts-...,"[0.01595756597816944, -0.0019251321209594607, ...",0.775186
105,Zu dieser Zeit war er auch Inhaber einer Tanks...,"[0.004819521214812994, -0.016626210883259773, ...",0.774874


In [174]:
res_20_df["Text"].iloc[1]

'Er erhielt diese Auszeichnung als erster Sportler.'